In [9]:
# add layer
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plot

def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    layer_name = 'layer%s' % n_layer
    with tf.name_scope('n_layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W') # captial for matrix
            tf.histogram_summary(layer_name + '/weights', Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b') # a list not a matrix
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)

        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
            tf.histogram_summary(layer_name + '/outputs', outputs)
        return outputs

x_data = np.linspace(-1, 1, 300)[:, np.newaxis] # 300 rows = 300 examples
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

# 圖的大框架
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input') # None for unlimited
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

layer1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu) # input layer
prediction = add_layer(layer1, 10, 1, n_layer=2, activation_function=None)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))
    tf.scalar_summary('loss', loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    #load into a document
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter("logs/", sess.graph)
    sess.run(init)
    
    fig = plot.figure()
    ax = fig.add_subplot(1,1,1)
    ax.scatter(x_data, y_data)
    plot.ion()
    plot.show()
    
    for i in range(1000):
        sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
        if i%50 == 0:
#             print(sess.run(loss, feed_dict={xs: x_data, ys:y_data}))
            try:
                 ax.lines.remove(lines[0])
            except Exception:
                pass
            prediction_value = sess.run(prediction, feed_dict={xs: x_data})
            lines = ax.plot(x_data, prediction_value, 'r-', lw=5)
            plot.pause(0.5)
            
            result = sess.run(merged, feed_dict={xs:x_data, ys:y_data})
            writer.add_summary(result, i)

AttributeError: module 'tensorflow' has no attribute 'histogram_summary'